In [16]:
%%pyspark project.spark.compatibility
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col, length, when
import re


df = spark.read.option("header", "true").csv("raw_path")

AnalysisException:[PATH_NOT_FOUND] Path does not exist: file:/tmp/sss.
Unable to run statement for connection: project.spark.compatibility. Error: Code execution failed


Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:08.853330 | Run duration : 0:00:03.641964s.


ExecutionException: Code execution failed

In [ ]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df.show()

In [17]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment

def clean_text_combined(text):
    if text is None:
        return ""
    
    try:
        # HTML 태그와 엔티티 처리
        text = re.sub(r'<br>', ' ', text)
        text = re.sub(r'</br>', ' ', text)
        text = re.sub(r'<br\s*/?>', ' ', text)
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'<[^&]*>', '', text)
        text = re.sub(r'&[a-zA-Z0-9#]+;', '', text)
        
        # URL, 전화번호, 괄호 내용 제거
        text = re.sub(r'http[s]?://\S+', '', text)
        text = re.sub(r'\d{4}-\d{4}', '', text)
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub(r'^\(광고\)', '', text)
        
        # 특정 키워드 포함 줄 제거
        lines = text.split('\n')
        filtered_lines = []
        for line in lines:
            if not any(keyword in line for keyword in ['무료수신거부', '준법', '광고']):
                filtered_lines.append(line)
        
        text = '\n'.join(filtered_lines)
        
        # 특수문자 제거
        special_chars = ['※', '◆', '■', '▶', '★', '☞', '☎', 'br', 'brbr']
        for char in special_chars:
            text = text.replace(char, '')
        
        # 연속된 공백과 줄바꿈 정리
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    except:
        return text


Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:16.039503 | Run duration : 0:00:03.437275s.


In [18]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
clean_text_combined_udf = udf(clean_text_combined, StringType())
df_final = df.withColumn("contents_final", clean_text_combined_udf(col("contents"))) \
            .filter(length(col("contents_final")) > 30)


Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:19.483518 | Run duration : 0:00:03.947500s.


In [19]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_cleaned_two=df_final.select('title', 'contents_final', 'label', 'frequency')


Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:23.439173 | Run duration : 0:00:03.540724s.


In [24]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_cleaned_two.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:49.366991 | Run duration : 0:00:00.894828s.


In [21]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered = df_cleaned_two.filter(col("label").isin(["Y", "N"]))


Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:34.560662 | Run duration : 0:00:03.552706s.


In [22]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered.count()


23
Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:38.118263 | Run duration : 0:00:07.171635s.


In [23]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered_final = df_filtered.filter(col("label").isin(["Y", "N"])) \
    .withColumn("label", when(col("label") == "Y", 1).otherwise(0)) \
    .withColumn("frequency", col("frequency").cast("int"))


Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:45.295863 | Run duration : 0:00:03.480622s.


In [25]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered_final.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 05:49:59.353000 | Run duration : 0:00:01.010064s.


In [11]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
# label별로 분리
df_label_0 = df_filtered_final.filter(col("label") == 0)
df_label_1 = df_filtered_final.filter(col("label") == 1)

# 각각 split
train_0, test_0 = df_label_0.randomSplit([0.7, 0.3], seed=42)
train_1, test_1 = df_label_1.randomSplit([0.7, 0.3], seed=42)

# 합치기
train_df = train_0.union(train_1).orderBy(rand(seed=42))
test_df = test_0.union(test_1).orderBy(rand(seed=42))


Connection: project.spark.compatibility | Run start time: 2025-12-10 05:27:16.591603 | Run duration : 0:00:03.373411s.


In [26]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
train_df.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 05:50:05.819591 | Run duration : 0:00:00.924274s.


In [27]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
test_df.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 05:50:11.671959 | Run duration : 0:00:01.019101s.


In [14]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
train_df.write.option("header",True) \
 .csv("train_path")



Connection: project.spark.compatibility | Run start time: 2025-12-10 05:41:28.892647 | Run duration : 0:00:11.944232s.


In [15]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
test_df.write.option("header",True) \
 .csv("test_path")



Connection: project.spark.compatibility | Run start time: 2025-12-10 05:41:40.842660 | Run duration : 0:00:08.879701s.


In [ ]:
%%pyspark default.spark
# Enter your code at the start of this line to replace this comment